In [2]:
from enum import Enum

import numpy as np
from numba import njit
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


@njit
def sequence(z, c, md):
    depth = 0
    while abs(z) <= 2 and depth < md:
        z = z * z + c
        depth += 1
    return depth


class FractalType(Enum):
    MANDELBROT = 1,
    JULIA = 2,
    BARNSLEY = 3


class FractalView:
    def __init__(self, w=1000, h=1000, depth=256, out='fractal'):
        self.width = w
        self.height = h
        self.depth = depth
        self.out = out

        self.__barnsley_transforms = [
            (lambda z: 0.16 * z.imag * 1j, 0.01),
            (lambda z: 0.85 * z.real + 0.04 * z.imag + 1j * (-0.04 * z.real + 0.85 * z.imag + 1.6), 0.85),
            (lambda z: 0.2 * z.real - 0.26 * z.imag + 1j * (0.23 * z.real + 0.22 * z.imag + 1.6), 0.07),
            (lambda z: -0.15 * z.real + 0.28 * z.imag + 1j * (0.26 * z.real + 0.24 * z.imag + 0.44), 0.07)
        ]

    def __point_prob(self, z, z_prob):
        for transform, prob in self.__barnsley_transforms:
            z_prob -= prob
            if z_prob <= 0:
                return transform(z)

    def __mandelbrot_fill(self, x, y, frac):
        for i in range(self.width):
            for j in range(self.height):
                frac[j, i] = sequence(0, x[i] + 1j * y[j], self.depth)

    def __julia_fill(self, x, y, c, frac):
        for i in range(self.width):
            for j in range(self.height):
                frac[j, i] = sequence(x[i] + 1j * y[j], c, self.depth)

    def __barnsley_fill(self, re_min, re_max, im_min, im_max, frac):
        z = 0 + 0j
        for _ in range(self.depth):
            z = self.__point_prob(z, np.random.random())

            i = int((z.real - re_min) / (re_max - re_min) * (self.width - 1))
            j = int((z.imag - im_min) / (im_max - im_min) * (self.height - 1))

            if i in range(0, self.width) and j in range(0, self.height):
                frac[j, i] += 1

    def __create_seq_frac(self, ftype, re_min, re_max, im_min, im_max, c=0):
        x = np.linspace(re_min, re_max, self.width)
        y = np.linspace(im_min, im_max, self.height)

        frac = np.zeros((self.height, self.width))

        self.__mandelbrot_fill(x, y, frac) if ftype == FractalType.MANDELBROT else self.__julia_fill(x, y, c, frac)

        return frac

    def __create_point_frac(self, ftype, re_min, re_max, im_min, im_max):
        frac = np.zeros((self.height, self.width))

        if ftype == FractalType.BARNSLEY:
            self.__barnsley_fill(re_min, re_max, im_min, im_max, frac)
            frac = np.log(frac + 1)

        return frac

    def __create_frac(self, ftype, re_min, re_max, im_min, im_max, c=0):
        if ftype == FractalType.MANDELBROT:
            return self.__create_seq_frac(FractalType.MANDELBROT, re_min, re_max, im_min, im_max)
        elif ftype == FractalType.JULIA:
            return self.__create_seq_frac(FractalType.JULIA, re_min, re_max, im_min, im_max, c)
        else:
            return self.__create_point_frac(FractalType.BARNSLEY, re_min, re_max, im_min, im_max)

    def __draw(self, frac):
        plt.imsave(f'{self.out}.png', frac, cmap='hot')

    def draw_mandelbrot(self, re_min=-2, re_max=1, im_min=-1.5, im_max=1.5):
        self.__draw(self.__create_frac(FractalType.MANDELBROT, re_min, re_max, im_min, im_max))

    def draw_julia(self, c=-0.5251993 + 1j * 0.5251993, re_min=-1.5, re_max=1.5, im_min=-1.5, im_max=1.5):
        self.__draw(self.__create_frac(FractalType.JULIA, re_min, re_max, im_min, im_max, c))

    def draw_barnsley(self, re_min=-3, re_max=3, im_min=-1, im_max=11):
        self.__draw(self.__create_frac(FractalType.BARNSLEY, re_min, re_max, im_min, im_max))

    def __mandelbrot_update(self, frame, image, tar):
        center_re = (self.upd_re_min + self.upd_re_max) / 2
        center_im = (self.upd_im_min + self.upd_im_max) / 2

        re_width = (self.upd_re_max - self.upd_re_min) * self.zoom
        im_height = (self.upd_im_max - self.upd_im_min) * self.zoom

        center_re += (tar.real - center_re) * (1 - self.zoom)
        center_im += (tar.imag - center_im) * (1 - self.zoom)

        self.upd_re_min, self.upd_re_max = center_re - re_width / 2, center_re + re_width / 2
        self.upd_im_min, self.upd_im_max = center_im - im_height / 2, center_im + im_height / 2

        image.set_array(self.__create_frac(FractalType.MANDELBROT, self.upd_re_min, self.upd_re_max,
                                           self.upd_im_min, self.upd_im_max))

        return [image]

    def __julia_update(self, frame, image, rad):
        alpha = (2 * np.pi * frame) / self.frames
        c = rad * (np.cos(alpha) + 1j * np.sin(alpha))
        image.set_array(self.__create_frac(FractalType.JULIA, self.upd_re_min, self.upd_re_max,
                                           self.upd_im_min, self.upd_im_max, c))
        return [image]

    def __animate(self, inp, ftype, re_min, re_max, im_min, im_max, frames, zoom=1):
        fig, ax = plt.subplots(figsize=(self.width / 100, self.height / 100))
        fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
        ax.axis('off')

        frac = self.__create_frac(ftype, re_min, re_max, im_min, im_max, inp)
        image = ax.imshow(frac, cmap='hot', animated=True)

        self.frames = frames
        self.zoom = zoom

        self.upd_re_min = re_min
        self.upd_re_max = re_max

        self.upd_im_min = im_min
        self.upd_im_max = im_max

        update = self.__mandelbrot_update if ftype == FractalType.MANDELBROT else self.__julia_update

        mandelbrot_anim = FuncAnimation(fig, update, fargs=(image, inp), frames=frames, interval=50, blit=True)
        mandelbrot_anim.save(f'{self.out}.gif', writer='pillow', fps=25)

        plt.close(fig)

    def animate_mandelbrot(self, tar=0.001643721971153 + 1j * 0.822467633258876, re_min=-2, re_max=1, 
                           im_min=-1.5, im_max=1.5, frames=1000, zoom=0.95):
        self.__animate(tar, FractalType.MANDELBROT, re_min, re_max, im_min, im_max, frames, zoom)
    
    def animate_julia(self, rad=0.7885, re_min=-1.5, re_max=1.5, im_min=-1.5, im_max=1.5, frames=500):
        self.__animate(rad, FractalType.JULIA, re_min, re_max, im_min, im_max, frames)


width, height = 1024, 1024
max_depth = 1024 ** 2

# c = -0.5251993 + 1j * 0.5251993

# rmin = -2
# rmax = 1
# 
# imin = -1.5
# imax = 1.5
# 
# center_re = (rmin + rmax) / 2
# center_im = (imin + imax) / 2
# 
# re_width = (rmax - rmin) * zoom
# im_height = (imax - imin) * zoom
# 
# center_re += (tar.real - center_re) * (1 - zoom)
# center_im += (tar.imag - center_im) * (1 - zoom)
# 
# rmin, rmax = center_re - re_width / 2, center_re + re_width / 2
# imin, imax = center_im - im_height / 2, center_im + im_height / 2

fv = FractalView(width, height, max_depth)

# fv.draw_mandelbrot(rmin, rmax, imin, imax)
# fv.draw_julia(-0.262+0.355j)
fv.draw_barnsley()

# fv.animate_mandelbrot()
# fv.animate_julia()